# Atop and Over `⍤⍥`

In this chapter, we'll cover the two compositional operators, [Atop](http://help.dyalog.com/latest/index.htm#Language/Primitive%20Operators/Atop.htm) (`⍤`) and [Over](http://help.dyalog.com/latest/index.htm#Language/Primitive%20Operators/Over.htm) (`⍥`), introduced in Dyalog version 18.0. 

## Atop

Atop has been assigned `function⍤function`, thus sharing the symbol with the rank operator's `function⍤array`. You should be familiar with the 2-train, which is also called "atop": `(f g)Y` and `X(f g)Y`. Maybe you've even been burned by `f∘g Y` being an atop, but `X f∘g Y` _not_ being an atop. Well, the atop operator is what you would expect, i.e. `f⍤g Y` is exactly like `f∘g Y` but `X f⍤g Y` is `f X g Y` or `X (f g) Y`. We strongly recommend transitioning to use `⍤` in places where you've hitherto used monadic `f∘g`: it will prevent (at least one potential cause of) frustration should you ever decide to add a left argument to your code. 

Let's say you define a function that returns the magnitude of reciprocal:

In [1]:
|∘÷ ¯4
|∘÷ ¯5

Rebuilding user command cache... 
done

0.25

0.2

(This could be written without the `∘`, but this is a very simple function useful for illustration purposes.)

Now you get a feature request that the function should take a left argument which is a numerator (instead of the default 1). 

In [2]:
2 |∘÷ ¯4
2 |∘÷ ¯5

1.75

1.8

Oops. However: 

In [3]:
|⍤÷ ¯4
|⍤÷ ¯5
2 |⍤÷ ¯4
2 |⍤÷ ¯5

0.25

0.2

0.5

0.4

One way to look at `f∘g` vs `f⍤g` is that, when given a left argument, `∘` gives it to the left-hand function and `⍤` gives it to the right-hand function. Other than that, they are equivalent.  Another way to look at `f∘g` vs `f⍤g` is simply choosing order of the first two tokens in the equivalent explicit expression: `X f∘g Y` computes `X f g Y` and `X f⍤g Y` computes `f X g Y`. So we're simply swapping `X` and `f`.

Then there's the classic problem with slashes, especially in tacit programming. If you've ever tried using replicate/compress in a train, you'll have bumped into the fact that slashes prefer being operators over being functions. This means that `{(5<⍵)/⍵}` doesn't convert to `(5<⊢)/⊢`.

While it may not be obvious at first sight, if we define `f←5<⊢` it might become clearer that `f/⊢` isn't at all what we want. Now, there's an axiom in APL that an operator cannot be an operand. (Shh, don't mention `∘.f`). This means that if a slash ends up in a situation where it has to be an operand, it will resort to being a function. You may even have noticed that constructs like `⊢(/⍨)5<⊢` work fine, though `⊢/⍨5<⊢` doesn't. This is because the `/` in isolation with the `⍨` is forced to become the operand of `⍨`. But since operators bind from the left, `⊢/` binds first, and so `⊢/⍨5<⊢` becomes `(⊢/)⍨5<⊢` or `(5<⊢)⊢/(5<⊢)` which is usually not what you want.

So, `⍤` to the rescue. If `⍤` (or any dyadic operator) is found to the immediate left of a slash, then clearly the dyadic operator cannot be the operand of the slash, `⍤` being a dyadic operator itself, and it can't be part of the function on the left, since it requires a right-operand, too. Therefore, the slash is forced to become a function. So `-⍤/` is the negation of the replicate: 

In [4]:
1 0 2-⍤/10 20 30

¯10 ¯30 ¯30

It is easy to think then that "oh, this is an atop, so I should be able to do this with parentheses too; `(f g)`" but that'd be a mistake: `(-/)` is just a normal minus-reduction. Since `f⍤g` is "atop" and `(f g)` is "atop", you might think they are interchangeable. 

Another mistake is to think: "if a slash is an operand, it'll be a function" and then think that `/∘⊢` would work like `⊢⍤/` by pre-processing the right argument with a no-op rather than post-processing the result with a no-op.

Let's say we have a two-element vector of a mask and some data, and you want to "apply" the mask to the data… Perhaps your instinct is to try

In [5]:
apply ← // 

but that gives an enclosed result, which isn't what we want:

In [6]:
]display apply (1 0 1)'abc'

┌──────┐
│ ┌→─┐ │
│ │ac│ │
│ └──┘ │
└∊─────┘

Instead we can do something like

In [8]:
apply ← ⊃⊢⍤//

In [9]:
]display apply (1 0 1)'abc'

┌→─┐
│ac│
└──┘

In fact, once `⊢⍤/` becomes a common pattern, you can actually help the reader of your code by using `⊢⍤/` so they don't have to consider if your slash is Replicate or Reduce. For example, if your code says `z←x/y` it might not be obvious what's going on. If you instead write `z←x⊢⍤/y` your reader knows exactly what you're doing. 

Another example. Given a string, replace every character with two copies of itself prefixed and suffixed by a space. For example, `'abc'` becomes `' aa  bb  cc '`. Yes, you can do this with regex. Please don't. 

In [11]:
(⊢ ⊢⍤⍀ ⍨0 2 0⍴⍨3×≢)'abc'

aa bb cc

## Over

Recall how `f∘g` preprocesses the right argument of `f` using `g`. One way to look at _Over_ is simply as preprocessing all arguments of `f` using `g`. All, as in both or the only. So again `f⍥g Y` is the same as `f⍤g Y` and `f∘g Y`. The difference is, again, when we do a dyadic application. While `X f∘g Y` is `X f(g Y)` we have `X f⍥g Y` be `(g X)f(g Y)`. This may seem like an overly involved operator, but really, the pattern of preprocessing both arguments comes up a lot. Once you start looking for it, you'll see it all over.

For example, a dyadic function computing the sum of absolute values of its arguments:

In [12]:
¯1 ¯2 3 4 +⍥| 2 3 ¯8 5

3 5 11 9

Given arguments which are vectors, which one has the smallest maximum? Return `¯1` if the left argument has the smallest maximum, `1` if the right one has, or `0` if they are equal.

In [13]:
¯1 ¯2 3 4 ×⍤-⍥(⌈/) 2 3 ¯8 5

¯1

Beautiful use of both Atop and Over. You can, of course, omit the `⍤` here, unless used inline. OK, how about this: write an alternative to replicate which can take arguments of equal shape, both with rank greater than 1, and replicates the corresponding elements. Since the result might otherwise be ragged, you have to return a vector. 

```apl
      (2 3⍴⍳6) YourFunction 2 3⍴⎕A
ABBCCCDDDDEEEEEFFFFFF
```

In [14]:
(2 3⍴⍳6) ⊢⍤/⍥, 2 3⍴⎕A

ABBCCCDDDDEEEEEFFFFFF

Also, in this case, you don't _need_ `⊢⍤`, but it is good for clarity, and necessary if used inline in a train. A golfing tip regarding `⍥`: you can sometimes use it to pre-process the left argument, when it is a no-op on the right. For example, `1≡⍥,≡,⍴`, only ravels the left argument, since the right argument already is a vector.